In [ ]:
import matplotlib.pyplot as plt
import torch
from torch.quasirandom import SobolEngine
import pandas as pd
import numpy as np
from alse.utils import identify_samples_which_satisfy_constraints, normalize, unnormalize
from mpl_toolkits import mplot3d
# from matplotlib import cm
# from alse.utils import read_excel
from alse.alse_workflow import alse
import copy

In [ ]:
def get_energy_density(v, p, area):
    return p/(v*area)


In [ ]:
# Velocity (200-3000), Power (300-2700), Spot Size (1-4)
bounds = torch.tensor([[200, 300, 1], [3000, 2700, 4]])
# initial_training_points = unnormalize(torch.rand(10, 3), bounds)

# v = initial_training_points[:, 0]/60
# p = initial_training_points[:, 1]
# area = (initial_training_points[:, 2]/2)**2 * np.pi

# df = pd.DataFrame(initial_training_points.numpy())
# energy_density = get_energy_density(v, p, area)
# df["energy_density"] = energy_density.numpy()
# df = df[~(df['energy_density'] <5)]
# df = df[~(df['energy_density'] >100)]  
# df.columns = ['V', 'P', 'spot size', 'energy density']
# df = df.round({'V': 0, 'P': 0, 'spot size': 1})
# df

In [ ]:
tkwargs = {
    "device": torch.device("cpu"),
    "dtype": torch.float,
}
def read_excel(file_path, x_names, y_names):
                                                        # NaN to 0
    consolidated_data = pd.read_excel(f"{file_path}").fillna(0).loc
    print(consolidated_data[0][0])
    # Input parameters
    input_param = []
    for xname in x_names:
        input_param.append(torch.tensor(consolidated_data[:, f"{xname}"]))

    # Output parameters
    output_param = []
    for yname in y_names:
        output_param.append((torch.tensor((consolidated_data[:, f"{yname}"]))).unsqueeze(-1))

    X = torch.stack(tuple(input_param), -1)
    return X.double(), *output_param
          # Need .double() !
          # Otherwise if sheet contains int can cause problem

In [ ]:
file_path = "../test_data/8_12_data.xlsx"
df = pd.read_excel(file_path).fillna(0)
x_names = ["V (mm/min)", "P (W)", "spot size (mm)"]
y_names = ["width (mm)", "powder_cap", "wth"]

input_param = []
for xname in x_names:
    input_param.append(torch.tensor(df[xname]))

output_param = []
for yname in y_names:
    output_param.append(torch.tensor(df[yname]).unsqueeze(-1))
X = torch.stack(tuple(input_param), -1)

In [ ]:
print(X.shape[1])

In [ ]:
    
X, width, pow_cap, wth = read_excel("../test_data/8_12_data.xlsx",["V (mm/min)", "P (W)", "spot size (mm)"], ["width (mm)", "powder_cap", "wth"])

# X, Y = read_excel("/home/jovyan/alse/test_data/8_4_data.xlsx",["V (mm/min)", "P (W)"], ["width (mm)", "powder_cap", "wth"])

bounds = torch.tensor([[200, 300, 1], [3000, 2700, 4]])
constraints = [("gt", 2.5), ("gt", 50), ("gt", 6)]

# algo = alse(X, bounds, [width, pow_cap, wth], constraints)
algo = alse(X, bounds, output_param, constraints)

algo.initialize_model(["reg", "reg", "reg"])

In [ ]:
new_pts = algo.next_test_points(1)
print(new_pts)

In [ ]:
df = pd.DataFrame(new_pts.numpy())
df

In [ ]:
%matplotlib notebook

N=50
x=np.linspace(bounds[0][0], bounds[1][0], N)
y=np.linspace(bounds[0][1], bounds[1][1], N)
z=np.linspace(bounds[0][2], bounds[1][2], N)

Xplt, Yplt, Zplt = np.meshgrid(x,y,z)
Xplt = torch.from_numpy(Xplt)
Yplt = torch.from_numpy(Yplt)
Zplt = torch.from_numpy(Zplt)

xplt = torch.stack(
    (
        torch.reshape(Xplt, (Xplt.shape[0] * Xplt.shape[1]* Xplt.shape[2],)),
        torch.reshape(Yplt, (Yplt.shape[0] * Yplt.shape[1]* Yplt.shape[2],)),
        torch.reshape(Zplt, (Zplt.shape[0] * Zplt.shape[1]* Zplt.shape[2],)),
    ),
    dim=1,
)
for model in algo.list_of_models:
    model.eval()
    

# Your 4dimension, only for example (use yours)
predicted_width = algo.list_of_models[0](normalize(xplt.float(), bounds)).loc.detach().reshape(N,N,N)
predicted_pow_cap = algo.list_of_models[1](normalize(xplt.float(), bounds)).loc.detach().reshape(N,N,N)
predicted_wth = algo.list_of_models[2](normalize(xplt.float(), bounds)).loc.detach().reshape(N,N,N)
est_width = predicted_width > 2.5
est_pow_cap = predicted_pow_cap > 0.5
est_wth = predicted_wth > 6
printable = (est_width & est_pow_cap & est_wth)

act_width = width > 2.5
act_pow_cap = pow_cap > 0.5
act_wth = wth > 6
act_printable = (act_width & act_pow_cap & act_wth)

In [ ]:
X[:,0][act_printable.squeeze()]

In [ ]:
%matplotlib notebook

N=50
x=np.linspace(bounds[0][0], bounds[1][0], N)
y=np.linspace(bounds[0][1], bounds[1][1], N)
z=np.linspace(bounds[0][2], bounds[1][2], N)

Xplt, Yplt, Zplt = np.meshgrid(x,y,z)
Xplt = torch.from_numpy(Xplt)
Yplt = torch.from_numpy(Yplt)
Zplt = torch.from_numpy(Zplt)

xplt = torch.stack(
    (
        torch.reshape(Xplt, (Xplt.shape[0] * Xplt.shape[1]* Xplt.shape[2],)),
        torch.reshape(Yplt, (Yplt.shape[0] * Yplt.shape[1]* Yplt.shape[2],)),
        torch.reshape(Zplt, (Zplt.shape[0] * Zplt.shape[1]* Zplt.shape[2],)),
    ),
    dim=1,
)
for model in algo.list_of_models:
    model.eval()
    

# Your 4dimension, only for example (use yours)
predicted_width = algo.list_of_models[0](normalize(xplt.float(), bounds)).loc.detach().reshape(N,N,N)
predicted_pow_cap = algo.list_of_models[1](normalize(xplt.float(), bounds)).loc.detach().reshape(N,N,N)
predicted_wth = algo.list_of_models[2](normalize(xplt.float(), bounds)).loc.detach().reshape(N,N,N)
est_width = predicted_width > 2.5
est_pow_cap = predicted_pow_cap > 0.5
est_wth = predicted_wth > 6
printable = (est_width & est_pow_cap & est_wth)

act_width = width > 2.5
act_pow_cap = pow_cap > 0.5
act_wth = wth > 6
act_printable = (act_width & act_pow_cap & act_wth)
# Creating figure
fig = plt.figure()
ax = plt.axes(projection="3d")

# Creating plot
ax.scatter3D(Xplt[printable], Yplt[printable], Zplt[printable], alpha=0.5, marker='.')
ax.scatter3D(X[:,0][act_printable.squeeze()], X[:,1][act_printable.squeeze()], X[:,2][act_printable.squeeze()], marker='x', color = 'g')
ax.scatter3D(X[:,0][~act_printable.squeeze()], X[:,1][~act_printable.squeeze()], X[:,2][~act_printable.squeeze()], marker='x', color = 'r')

ax.set_xlabel("velocity (mm/min)")
ax.set_ylabel("power (W)")
ax.set_zlabel("spot size (mm)")
plt.show()

In [ ]:
predicted_width = algo.list_of_models[0](normalize(xplt.float(), bounds)).loc.detach()
predicted_pow_cap = algo.list_of_models[1](normalize(xplt.float(), bounds)).loc.detach()
predicted_wth = algo.list_of_models[2](normalize(xplt.float(), bounds)).loc.detach()

uncertainty = 0.03
desired_width1 = predicted_width > 2.75*(1-uncertainty)
desired_width2 = predicted_width < 2.75*(1+uncertainty)

desired_wth1 = predicted_wth > (2.75/0.2)*(1-uncertainty)
desired_wth2 = predicted_wth < (2.75/0.2)*(1+uncertainty)

printable = (desired_width1 & desired_width2 & desired_wth1 & desired_wth2)
desired_test_point = xplt[printable]
# xplt = torch.stack(
#     (
#         torch.reshape(Xplt, (Xplt.shape[0] * Xplt.shape[1]* Xplt.shape[2],)),
#         torch.reshape(Yplt, (Yplt.shape[0] * Yplt.shape[1]* Yplt.shape[2],)),
#         torch.reshape(Zplt, (Zplt.shape[0] * Zplt.shape[1]* Zplt.shape[2],)),
#     ),
#     dim=1,
# )

In [ ]:
df = pd.DataFrame(desired_test_point.numpy())
df.columns = ['V', 'P', 'spot size']
df['predicted_width'] = predicted_width[printable]
df['predicted_height'] = predicted_width[printable]/predicted_wth[printable]
df = df.round({'V': -1, 'P': 0, 'predicted_width':2, 'spot size': 3, 'predicted_height':2})

df

Accuracy Demo

In [ ]:
'''
Code below is for demo of the method
Actual function has been implemented
in alse_workflow.py
'''
uncertainty = 0.1

a1 = predicted_width > constraints[0][1]*(1-uncertainty)
a2 = predicted_width > constraints[0][1]*(1+uncertainty)

b1 = predicted_pow_cap > constraints[1][1]*(1-uncertainty)
b2 = predicted_pow_cap > constraints[1][1]*(1+uncertainty)

c1 = predicted_wth > constraints[2][1]*(1-uncertainty)
c2 = predicted_wth > constraints[2][1]*(1+uncertainty)

overlap1 = a1 & b1 & c1
overlap2 = ~(a2 & b2 & c2)
mask = (overlap1 & overlap2).unsqueeze(-1)

thick_overlap = torch.masked_select(xplt, mask).reshape(mask.sum(), xplt.shape[1])

In [ ]:
def get_test_points(candidates, num_points):
    return candidates[torch.randint(candidates.shape[0], (num_points,))]

test_points = get_test_points(thick_overlap, 10)
test_points

In [ ]:
import plotly.graph_objects as go
import plotly.express as px
'''
  *Proof that the above method works
   Overlap is a surface with thickness
'''
fig = go.Figure(data=[go.Scatter3d(x=thick_overlap[:,0], y=thick_overlap[:,1], z=thick_overlap[:,2], mode='markers', marker=dict(size = 2)),
                      go.Scatter3d(x=test_points[:,0], y=test_points[:,1], z=test_points[:,2],mode='markers', marker=dict(color='green', size = 5))])

fig.update_layout(
    autosize=False,
    width=1000,
    height=1000,
    title = go.layout.Title(text='Overlap'),
    scene=dict(xaxis_title='Velocity', 
                yaxis_title='Power',
                zaxis_title='Spot Size',
    xaxis = dict(nticks=5, range=[200,3000],),
    yaxis = dict(nticks=5, range=[300,2700],),
    zaxis = dict(nticks=5, range=[1,4],),)
)
fig.show()

In [ ]:
algo.get_test_points(num_points=10, uncertainty=0.1)